In [1]:
import pandas as pd
import numpy as np

#useful for displaying wide data frames
pd.set_option('display.max_columns', 50)

In [2]:
df_market = pd.read_csv("input/markets.csv")
df_runners = pd.read_csv("input/runners.csv",dtype={'barrier': np.int16,'handicap_weight': np.float16})


In [3]:
df_runners_and_market = pd.merge(df_runners, df_market,left_on='market_id',right_on='id',how='inner')
df_runners_and_market.index = df_runners_and_market['id_x'] 


In [4]:
forms = pd.read_csv('input/forms.csv')

In [5]:
df_runners_and_market = df_runners_and_market.join(forms[['overall_starts','overall_wins', 'overall_places', 'track_starts', 'track_wins']], on='horse_id')

In [6]:
df_runners_and_market['win_pct'] = df_runners_and_market['overall_wins'] / df_runners_and_market['overall_starts']
df_runners_and_market['place_pct'] = df_runners_and_market['overall_places'] / df_runners_and_market['overall_starts']
df_runners_and_market['win_pct'] = df_runners_and_market['win_pct'].replace('nan', 0)
df_runners_and_market['place_pct'] = df_runners_and_market['place_pct'].replace('nan', 0)

In [33]:
df_runners_and_market.drop('id_x', axis=1).to_csv('input/modified_data.csv', header=True, index=True)

In [80]:
mod = pd.read_csv('input/modified_data.csv', index_col='id_x')

In [81]:
mod.head()

,collected_at,market_id,position,place_paid,margin,horse_id,trainer_id,rider_id,handicap_weight,number,barrier,blinkers,emergency,form_rating_one,form_rating_two,form_rating_three,last_five_starts,favourite_odds_win,favourite_odds_place,favourite_pool_win,favourite_pool_place,tip_one_win,tip_one_place,tip_two_win,tip_two_place,...,tip_seven_place,tip_eight_win,tip_eight_place,tip_nine_win,tip_nine_place,id_y,timezone,venue_id,race_number,distance,condition_id,weather_id,total_pool_one_win,total_pool_one_place,total_pool_two_win,total_pool_two_place,total_pool_three_win,total_pool_three_place,overall_starts,overall_wins,overall_places,track_starts,track_wins,win_pct,place_pct
id_x,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,2016-06-26 18:54:31.800293,1,NaN,1,NaN,4,4.0,4.0,58.5,4,10,f,f,82.0,82.0,14.0,x80x2,f,f,f,f,f,NaN,f,t,...,t,f,t,t,t,1,2016-06-26 19:00:00,1,3,1200,1.0,1.0,29718.08,NaN,11564.5,5373.0,23464.8,5373.0,1,0,0,0,0,0.0,0.00
10,2016-06-26 18:54:31.974395,1,NaN,0,NaN,10,4.0,10.0,56.5,10,11,f,f,100.0,100.0,18.0,22x35,f,f,f,f,f,NaN,f,f,...,f,f,f,f,f,1,2016-06-26 19:00:00,1,3,1200,1.0,1.0,29718.08,NaN,11564.5,5373.0,23464.8,5373.0,1,0,0,0,0,0.0,0.00
5,2016-06-26 18:54:31.835329,1,NaN,0,NaN,5,5.0,5.0,56.5,5,5,f,f,76.0,76.0,0.0,f7,f,f,f,f,f,NaN,f,f,...,f,f,f,f,f,1,2016-06-26 19:00:00,1,3,1200,1.0,1.0,29718.08,NaN,11564.5,5373.0,23464.8,5373.0,4,0,1,1,0,0.0,0.25
6,2016-06-26 18:54:31.873492,1,NaN,0,NaN,6,6.0,6.0,56.5,6,12,f,f,85.0,85.0,2.0,f6462,f,f,f,f,f,NaN,f,f,...,f,f,f,f,f,1,2016-06-26 19:00:00,1,3,1200,1.0,1.0,29718.08,NaN,11564.5,5373.0,23464.8,5373.0,1,0,0,0,0,0.0,0.00
1,2016-06-26 18:54:31.746854,1,NaN,0,NaN,1,1.0,1.0,58.5,1,8,f,f,89.0,89.0,0.0,34x0x,f,f,f,f,f,NaN,f,f,...,f,f,f,f,f,1,2016-06-26 19:00:00,1,3,1200,1.0,1.0,29718.08,NaN,11564.5,5373.0,23464.8,5373.0,1,0,1,0,0,0.0,1.00


In [82]:
#want to get avg competition place_pct and win_pct
grouped =  df_runners_and_market.groupby('market_id')

In [72]:
one = grouped.get_group(1)
two = one = grouped.get_group(2)
groups = [(1, one), (2,two)]

In [96]:
competitionDf = pd.DataFrame(columns=['horse_id', 'market_id', 'comp_win_pct', 'comp_place_pct'])
for mkt, group in grouped:
    group_comp_df = pd.DataFrame(columns=['horse_id', 'market_id', 'comp_win_pct', 'comp_place_pct'])
    for i, row in group.iterrows():
        group_comp_df = group_comp_df.append({'horse_id': row.horse_id, 'market_id': mkt, 'comp_win_pct': group.drop(i).win_pct.mean(), 
                                  'comp_place_pct': group.drop(i).place_pct.mean()}, ignore_index=True)
    competitionDf = pd.concat([competitionDf, group_comp_df])

In [98]:
competitionDf.head()

,horse_id,market_id,comp_win_pct,comp_place_pct
0,4.0,1.0,0.076923,0.212056
1,10.0,1.0,0.076923,0.212056
2,5.0,1.0,0.076923,0.192825
3,6.0,1.0,0.076923,0.212056
4,1.0,1.0,0.076923,0.135133


In [99]:
modWithComp = pd.merge(mod,competitionDf, left_on=['horse_id', 'market_id'], right_on=['horse_id', 'market_id'])

In [101]:
modWithComp.head()

,collected_at,market_id,position,place_paid,margin,horse_id,trainer_id,rider_id,handicap_weight,number,barrier,blinkers,emergency,form_rating_one,form_rating_two,form_rating_three,last_five_starts,favourite_odds_win,favourite_odds_place,favourite_pool_win,favourite_pool_place,tip_one_win,tip_one_place,tip_two_win,tip_two_place,...,tip_eight_place,tip_nine_win,tip_nine_place,id_y,timezone,venue_id,race_number,distance,condition_id,weather_id,total_pool_one_win,total_pool_one_place,total_pool_two_win,total_pool_two_place,total_pool_three_win,total_pool_three_place,overall_starts,overall_wins,overall_places,track_starts,track_wins,win_pct,place_pct,comp_win_pct,comp_place_pct
0,2016-06-26 18:54:31.800293,1,NaN,1,NaN,4,4.0,4.0,58.5,4,10,f,f,82.0,82.0,14.0,x80x2,f,f,f,f,f,NaN,f,t,...,t,t,t,1,2016-06-26 19:00:00,1,3,1200,1.0,1.0,29718.08,NaN,11564.5,5373.0,23464.8,5373.0,1,0,0,0,0,0.0,0.00,0.076923,0.212056
1,2016-06-26 18:54:31.974395,1,NaN,0,NaN,10,4.0,10.0,56.5,10,11,f,f,100.0,100.0,18.0,22x35,f,f,f,f,f,NaN,f,f,...,f,f,f,1,2016-06-26 19:00:00,1,3,1200,1.0,1.0,29718.08,NaN,11564.5,5373.0,23464.8,5373.0,1,0,0,0,0,0.0,0.00,0.076923,0.212056
2,2016-06-26 18:54:31.835329,1,NaN,0,NaN,5,5.0,5.0,56.5,5,5,f,f,76.0,76.0,0.0,f7,f,f,f,f,f,NaN,f,f,...,f,f,f,1,2016-06-26 19:00:00,1,3,1200,1.0,1.0,29718.08,NaN,11564.5,5373.0,23464.8,5373.0,4,0,1,1,0,0.0,0.25,0.076923,0.192825
3,2016-06-26 18:54:31.873492,1,NaN,0,NaN,6,6.0,6.0,56.5,6,12,f,f,85.0,85.0,2.0,f6462,f,f,f,f,f,NaN,f,f,...,f,f,f,1,2016-06-26 19:00:00,1,3,1200,1.0,1.0,29718.08,NaN,11564.5,5373.0,23464.8,5373.0,1,0,0,0,0,0.0,0.00,0.076923,0.212056
4,2016-06-26 18:54:31.746854,1,NaN,0,NaN,1,1.0,1.0,58.5,1,8,f,f,89.0,89.0,0.0,34x0x,f,f,f,f,f,NaN,f,f,...,f,f,f,1,2016-06-26 19:00:00,1,3,1200,1.0,1.0,29718.08,NaN,11564.5,5373.0,23464.8,5373.0,1,0,1,0,0,0.0,1.00,0.076923,0.135133


In [110]:
modWithComp.to_csv('input/modWithComp.csv', index=False)

In [111]:
modRead = pd.read_csv('input/modWithComp.csv')

In [112]:
modRead.head()

,collected_at,market_id,position,place_paid,margin,horse_id,trainer_id,rider_id,handicap_weight,number,barrier,blinkers,emergency,form_rating_one,form_rating_two,form_rating_three,last_five_starts,favourite_odds_win,favourite_odds_place,favourite_pool_win,favourite_pool_place,tip_one_win,tip_one_place,tip_two_win,tip_two_place,...,tip_eight_place,tip_nine_win,tip_nine_place,id_y,timezone,venue_id,race_number,distance,condition_id,weather_id,total_pool_one_win,total_pool_one_place,total_pool_two_win,total_pool_two_place,total_pool_three_win,total_pool_three_place,overall_starts,overall_wins,overall_places,track_starts,track_wins,win_pct,place_pct,comp_win_pct,comp_place_pct
0,2016-06-26 18:54:31.800293,1,NaN,1,NaN,4,4.0,4.0,58.5,4,10,f,f,82.0,82.0,14.0,x80x2,f,f,f,f,f,NaN,f,t,...,t,t,t,1,2016-06-26 19:00:00,1,3,1200,1.0,1.0,29718.08,NaN,11564.5,5373.0,23464.8,5373.0,1,0,0,0,0,0.0,0.00,0.076923,0.212056
1,2016-06-26 18:54:31.974395,1,NaN,0,NaN,10,4.0,10.0,56.5,10,11,f,f,100.0,100.0,18.0,22x35,f,f,f,f,f,NaN,f,f,...,f,f,f,1,2016-06-26 19:00:00,1,3,1200,1.0,1.0,29718.08,NaN,11564.5,5373.0,23464.8,5373.0,1,0,0,0,0,0.0,0.00,0.076923,0.212056
2,2016-06-26 18:54:31.835329,1,NaN,0,NaN,5,5.0,5.0,56.5,5,5,f,f,76.0,76.0,0.0,f7,f,f,f,f,f,NaN,f,f,...,f,f,f,1,2016-06-26 19:00:00,1,3,1200,1.0,1.0,29718.08,NaN,11564.5,5373.0,23464.8,5373.0,4,0,1,1,0,0.0,0.25,0.076923,0.192825
3,2016-06-26 18:54:31.873492,1,NaN,0,NaN,6,6.0,6.0,56.5,6,12,f,f,85.0,85.0,2.0,f6462,f,f,f,f,f,NaN,f,f,...,f,f,f,1,2016-06-26 19:00:00,1,3,1200,1.0,1.0,29718.08,NaN,11564.5,5373.0,23464.8,5373.0,1,0,0,0,0,0.0,0.00,0.076923,0.212056
4,2016-06-26 18:54:31.746854,1,NaN,0,NaN,1,1.0,1.0,58.5,1,8,f,f,89.0,89.0,0.0,34x0x,f,f,f,f,f,NaN,f,f,...,f,f,f,1,2016-06-26 19:00:00,1,3,1200,1.0,1.0,29718.08,NaN,11564.5,5373.0,23464.8,5373.0,1,0,1,0,0,0.0,1.00,0.076923,0.135133
